In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
%matplotlib inline

In [2]:
members = pd.read_csv('data/NSS_GreatCare_Members.csv')
admissions = pd.read_csv('data/NSS_Hospital_Inpatient_Admissions.csv')
primary = pd.read_csv('data/NSS_PrimaryCare_Office_Visits.csv')

C:\Users\rhilb\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
pd.set_option("display.max_columns", None)

In [4]:
admissions.head()

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,0,0,1.0,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,0,0,3.0,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,1,0,1.0,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,0,0,3.0,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,0,0,2.0,2,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0


In [5]:
hospital_visits_by_patient = admissions['Patient ID'].value_counts()
hospital_visits_by_patient

DS1|3812     17
DS1|26751    15
DS1|53241    13
DS1|10451    12
DS1|12422    11
             ..
DS1|31911     1
DS1|36994     1
DS1|48301     1
DS1|55779     1
DS1|35119     1
Name: Patient ID, Length: 4701, dtype: int64

In [6]:
hospital_visits_by_patient.max()

17

In [7]:
# change to datetimes
admissions['Encounter Start Date'] = pd.to_datetime(admissions['Encounter Start Date'])
admissions['Encounter End Date'] = pd.to_datetime(admissions['Encounter End Date'])
admissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,0,0,1.0,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,0,0,3.0,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,1,0,1.0,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,0,0,3.0,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,0,0,2.0,2,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,0,0,2.0,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,0,1,3.0,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,0,0,1.0,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,0,0,2.0,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0


In [8]:
# sort by patient id and encounter start date so that repeated patients are stacked in order
admissions.sort_values(["Patient ID", 'Encounter Start Date'], inplace=True)
admissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
117,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1962461012,1.962461e+09,1710918545,2013-06-13,2013-06-16,1,0,1.0,1,1,21.0,NaN,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,29219.00,250.0
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,0,0,1.0,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,0,0,3.0,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,1,0,1.0,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,0,0,3.0,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,0,0,2.0,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,0,1,3.0,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,0,0,1.0,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,0,0,2.0,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0


In [9]:
# calculate the difference between start & end dates for each row to the row above it to find number of days between start dates
# this will include several values that are unhelpful because if a row does not represent a readmission, this difference
# will be nonsensical, but those values will be dropped later
admissions['start_date_diff'] = admissions['Encounter Start Date'].diff()
admissions['end_date_diff'] = admissions['Encounter End Date'].diff()
admissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,start_date_diff,end_date_diff
117,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1962461012,1.962461e+09,1710918545,2013-06-13,2013-06-16,1,0,1.0,1,1,21.0,NaN,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,29219.00,250.0,NaT,NaT
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,0,0,1.0,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,-152 days,-153 days
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,0,0,3.0,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,358 days,363 days
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,1,0,1.0,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,-571 days,-570 days
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,0,0,3.0,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,176 days,171 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,0,0,2.0,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,-207 days,-206 days
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,0,1,3.0,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,-183 days,-185 days
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,0,0,1.0,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,471 days,470 days
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,0,0,2.0,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,3 days,4 days


In [10]:
# calculate the length of stay for each hospital visit
admissions['length_of_stay'] = admissions['Encounter End Date'] - admissions['Encounter Start Date']
admissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,start_date_diff,end_date_diff,length_of_stay
117,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1962461012,1.962461e+09,1710918545,2013-06-13,2013-06-16,1,0,1.0,1,1,21.0,NaN,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,29219.00,250.0,NaT,NaT,3 days
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,0,0,1.0,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,-152 days,-153 days,2 days
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,0,0,3.0,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,358 days,363 days,7 days
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,1,0,1.0,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,-571 days,-570 days,8 days
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,0,0,3.0,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,176 days,171 days,3 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,0,0,2.0,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,-207 days,-206 days,5 days
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,0,1,3.0,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,-183 days,-185 days,3 days
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,0,0,1.0,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,471 days,470 days,2 days
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,0,0,2.0,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,3 days,4 days,3 days


In [11]:
# calculate the length between end date and start date
admissions['length_between'] = admissions['end_date_diff'] - admissions['length_of_stay']
admissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,start_date_diff,end_date_diff,length_of_stay,length_between
117,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1962461012,1.962461e+09,1710918545,2013-06-13,2013-06-16,1,0,1.0,1,1,21.0,NaN,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,29219.00,250.0,NaT,NaT,3 days,NaT
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,0,0,1.0,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,-152 days,-153 days,2 days,-155 days
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,0,0,3.0,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,358 days,363 days,7 days,356 days
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,1,0,1.0,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,-571 days,-570 days,8 days,-578 days
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,0,0,3.0,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,176 days,171 days,3 days,168 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,0,0,2.0,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,-207 days,-206 days,5 days,-211 days
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,0,1,3.0,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,-183 days,-185 days,3 days,-188 days
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,0,0,1.0,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,471 days,470 days,2 days,468 days
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,0,0,2.0,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,3 days,4 days,3 days,1 days


In [12]:
# clean patient ID
patient_id = admissions['Patient ID'].str.split('|', expand=True)
patient_id

,0,1
117,DS1,10
0,DS1,10033
1,DS1,10043
2,DS1,10073
3,DS1,10101
...,...,...
6208,DS1,9979
6209,DS1,9981
6210,DS1,9994
6211,DS1,9994


In [13]:
admissions['patient_id_clean'] = patient_id[1]
admissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,start_date_diff,end_date_diff,length_of_stay,length_between,patient_id_clean
117,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1962461012,1.962461e+09,1710918545,2013-06-13,2013-06-16,1,0,1.0,1,1,21.0,NaN,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,29219.00,250.0,NaT,NaT,3 days,NaT,10
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,0,0,1.0,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,-152 days,-153 days,2 days,-155 days,10033
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,0,0,3.0,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,358 days,363 days,7 days,356 days,10043
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,1,0,1.0,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,-571 days,-570 days,8 days,-578 days,10073
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,0,0,3.0,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,176 days,171 days,3 days,168 days,10101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,0,0,2.0,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,-207 days,-206 days,5 days,-211 days,9979
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,0,1,3.0,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,-183 days,-185 days,3 days,-188 days,9981
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,0,0,1.0,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,471 days,470 days,2 days,468 days,9994
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,0,0,2.0,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,3 days,4 days,3 days,1 days,9994


In [14]:
admissions['patient_id_clean'] = pd.to_numeric(admissions['patient_id_clean'])

In [15]:
# calculate difference between patient ID for subsequent rows 
# anytime this value is nonzero will not be a readmission since it is sorted by patient ID
admissions['patient_id_diff'] = admissions['patient_id_clean'].diff()
admissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,start_date_diff,end_date_diff,length_of_stay,length_between,patient_id_clean,patient_id_diff
117,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1962461012,1.962461e+09,1710918545,2013-06-13,2013-06-16,1,0,1.0,1,1,21.0,NaN,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,29219.00,250.0,NaT,NaT,3 days,NaT,10,NaN
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,0,0,1.0,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,-152 days,-153 days,2 days,-155 days,10033,10023.0
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,0,0,3.0,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,358 days,363 days,7 days,356 days,10043,10.0
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,1,0,1.0,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,-571 days,-570 days,8 days,-578 days,10073,30.0
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,0,0,3.0,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,176 days,171 days,3 days,168 days,10101,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,0,0,2.0,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,-207 days,-206 days,5 days,-211 days,9979,1.0
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,0,1,3.0,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,-183 days,-185 days,3 days,-188 days,9981,2.0
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,0,0,1.0,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,471 days,470 days,2 days,468 days,9994,13.0
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,0,0,2.0,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,3 days,4 days,3 days,1 days,9994,0.0


In [16]:
# create 0 day and 30 day datetime columns b/c I don't know how to compare datetimes to integers
admissions['ex_date1'] = '2012-09-01'
admissions['ex_date1'] = pd.to_datetime(admissions['ex_date1'])
admissions['ex_date2'] = '2012-10-01'
admissions['ex_date2'] = pd.to_datetime(admissions['ex_date2'])
admissions['thirty_days'] = admissions['ex_date2'] - admissions['ex_date1']
admissions['zero_days'] = admissions['ex_date1'] - admissions['ex_date1']
admissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,start_date_diff,end_date_diff,length_of_stay,length_between,patient_id_clean,patient_id_diff,ex_date1,ex_date2,thirty_days,zero_days
117,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1962461012,1.962461e+09,1710918545,2013-06-13,2013-06-16,1,0,1.0,1,1,21.0,NaN,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,29219.00,250.0,NaT,NaT,3 days,NaT,10,NaN,2012-09-01,2012-10-01,30 days,0 days
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,0,0,1.0,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,-152 days,-153 days,2 days,-155 days,10033,10023.0,2012-09-01,2012-10-01,30 days,0 days
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,0,0,3.0,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,358 days,363 days,7 days,356 days,10043,10.0,2012-09-01,2012-10-01,30 days,0 days
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,1,0,1.0,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,-571 days,-570 days,8 days,-578 days,10073,30.0,2012-09-01,2012-10-01,30 days,0 days
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,0,0,3.0,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,176 days,171 days,3 days,168 days,10101,28.0,2012-09-01,2012-10-01,30 days,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,0,0,2.0,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,-207 days,-206 days,5 days,-211 days,9979,1.0,2012-09-01,2012-10-01,30 days,0 days
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,0,1,3.0,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,-183 days,-185 days,3 days,-188 days,9981,2.0,2012-09-01,2012-10-01,30 days,0 days
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,0,0,1.0,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,471 days,470 days,2 days,468 days,9994,13.0,2012-09-01,2012-10-01,30 days,0 days
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,0,0,2.0,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,3 days,4 days,3 days,1 days,9994,0.0,2012-09-01,2012-10-01,30 days,0 days


In [17]:
# filter out cases where length between is not between 0 and 30
readmissions = admissions[admissions['length_between'] >= admissions['zero_days']]
readmissions = readmissions[readmissions['length_between'] <= readmissions['thirty_days']]
# filter out cases where it is not readmission b/c patient_id_diff doesn't equal 0
readmissions = readmissions[readmissions['patient_id_diff'] == 0]
readmissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,start_date_diff,end_date_diff,length_of_stay,length_between,patient_id_clean,patient_id_diff,ex_date1,ex_date2,thirty_days,zero_days
45,HIA|DS1|10451|DS1|NPI|1902844988|1,DS1|10451,DS1|1,DS1|5047X,DS1|NPI|1902844988,DS1|NPI|1811123151,1.811123e+09,1902844988,2013-03-27,2013-04-16,1,1,1.0,1,70,21.0,571.1,Acute alcoholic hepatitis,NaN,NaN,0,17978.94,250.0,7 days,24 days,20 days,4 days,10451,0.0,2012-09-01,2012-10-01,30 days,0 days
37,HIA|DS1|10451|DS1|NPI|1427055839|1,DS1|10451,DS1|1,DS1|5047X,DS1|NPI|1427055839,DS1|NPI|1427344951,1.427345e+09,1427055839,2013-04-16,2013-04-26,0,0,2.0,4,6,21.0,571.2,Alcoholic cirrhosis of liver,NaN,NaN,0,54450.00,50.0,20 days,10 days,10 days,0 days,10451,0.0,2012-09-01,2012-10-01,30 days,0 days
38,HIA|DS1|10451|DS1|NPI|1427055839|2,DS1|10451,DS1|1,DS1|5047X,DS1|NPI|1427055839,DS1|NPI|1073566931,1.073567e+09,1427055839,2013-05-01,2013-05-09,1,0,1.0,1,6,23.0,571.2,Alcoholic cirrhosis of liver,NaN,NaN,0,51700.00,0.0,15 days,13 days,8 days,5 days,10451,0.0,2012-09-01,2012-10-01,30 days,0 days
40,HIA|DS1|10451|DS1|NPI|1427055839|4,DS1|10451,DS1|1,DS1|5047X,DS1|NPI|1427055839,DS1|NPI|1679541775,1.679542e+09,1427055839,2013-07-26,2013-08-03,0,0,1.0,1,6,21.0,571.2,Alcoholic cirrhosis of liver,NaN,NaN,0,59395.00,0.0,31 days,33 days,8 days,25 days,10451,0.0,2012-09-01,2012-10-01,30 days,0 days
41,HIA|DS1|10451|DS1|NPI|1427055839|5,DS1|10451,DS1|1,DS1|5047X,DS1|NPI|1427055839,DS1|NPI|1215189550,1.215190e+09,1427055839,2013-08-31,2013-09-02,0,0,3.0,2,1,21.0,571.2,Alcoholic cirrhosis of liver,NaN,NaN,0,27653.05,0.0,36 days,30 days,2 days,28 days,10451,0.0,2012-09-01,2012-10-01,30 days,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6185,HIA|DS1|9809|DS1|NPI|1689608150|2,DS1|9809,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1821297797,1.821298e+09,1689608150,2013-01-20,2013-01-24,0,0,2.0,1,1,21.0,NaN,NaN,I48.0,Paroxysmal atrial fibrillation,0,10721.07,250.0,11 days,13 days,4 days,9 days,9809,0.0,2012-09-01,2012-10-01,30 days,0 days
6199,HIA|DS1|9935|DS1|NPI|1902803315|1,DS1|9935,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1972545085,1.972545e+09,1902803315,2011-01-25,2011-01-26,0,0,2.0,4,62,21.0,NaN,NaN,I63.211,Cerebral infarction due to unspecified occlusi...,0,13596.10,250.0,3 days,2 days,1 days,1 days,9935,0.0,2012-09-01,2012-10-01,30 days,0 days
6197,HIA|DS1|9935|DS1|NPI|1366401333|1,DS1|9935,DS1|1,DS1|5047X,DS1|NPI|1366401333,DS1|NPI|1790917086,1.790917e+09,1366401333,2011-01-26,2011-02-07,0,0,2.0,D,6,21.0,NaN,NaN,I63.211,Cerebral infarction due to unspecified occlusi...,0,45890.00,250.0,1 days,12 days,12 days,0 days,9935,0.0,2012-09-01,2012-10-01,30 days,0 days
6202,HIA|DS1|9943|DS1|NPI|1184722779|2,DS1|9943,DS1|1,DS1|5047X,DS1|NPI|1184722779,DS1|NPI|1942320288,1.942320e+09,1184722779,2013-10-25,2013-10-28,1,0,1.0,1,6,21.0,NaN,NaN,K91.89,Other postprocedural complications and disorde...,0,12277.13,0.0,24 days,12 days,3 days,9 days,9943,0.0,2012-09-01,2012-10-01,30 days,0 days


In [18]:
# calculate readmission percentage
readmission_percentage = 100 * len(readmissions) / len(admissions)
readmission_percentage

11.26669885723483

In [68]:
len(readmissions)

700

In [19]:
# add column to readmissions that is just one to identify that row as a readmission
readmissions['readmission'] = 1
readmissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,start_date_diff,end_date_diff,length_of_stay,length_between,patient_id_clean,patient_id_diff,ex_date1,ex_date2,thirty_days,zero_days,readmission
45,HIA|DS1|10451|DS1|NPI|1902844988|1,DS1|10451,DS1|1,DS1|5047X,DS1|NPI|1902844988,DS1|NPI|1811123151,1.811123e+09,1902844988,2013-03-27,2013-04-16,1,1,1.0,1,70,21.0,571.1,Acute alcoholic hepatitis,NaN,NaN,0,17978.94,250.0,7 days,24 days,20 days,4 days,10451,0.0,2012-09-01,2012-10-01,30 days,0 days,1
37,HIA|DS1|10451|DS1|NPI|1427055839|1,DS1|10451,DS1|1,DS1|5047X,DS1|NPI|1427055839,DS1|NPI|1427344951,1.427345e+09,1427055839,2013-04-16,2013-04-26,0,0,2.0,4,6,21.0,571.2,Alcoholic cirrhosis of liver,NaN,NaN,0,54450.00,50.0,20 days,10 days,10 days,0 days,10451,0.0,2012-09-01,2012-10-01,30 days,0 days,1
38,HIA|DS1|10451|DS1|NPI|1427055839|2,DS1|10451,DS1|1,DS1|5047X,DS1|NPI|1427055839,DS1|NPI|1073566931,1.073567e+09,1427055839,2013-05-01,2013-05-09,1,0,1.0,1,6,23.0,571.2,Alcoholic cirrhosis of liver,NaN,NaN,0,51700.00,0.0,15 days,13 days,8 days,5 days,10451,0.0,2012-09-01,2012-10-01,30 days,0 days,1
40,HIA|DS1|10451|DS1|NPI|1427055839|4,DS1|10451,DS1|1,DS1|5047X,DS1|NPI|1427055839,DS1|NPI|1679541775,1.679542e+09,1427055839,2013-07-26,2013-08-03,0,0,1.0,1,6,21.0,571.2,Alcoholic cirrhosis of liver,NaN,NaN,0,59395.00,0.0,31 days,33 days,8 days,25 days,10451,0.0,2012-09-01,2012-10-01,30 days,0 days,1
41,HIA|DS1|10451|DS1|NPI|1427055839|5,DS1|10451,DS1|1,DS1|5047X,DS1|NPI|1427055839,DS1|NPI|1215189550,1.215190e+09,1427055839,2013-08-31,2013-09-02,0,0,3.0,2,1,21.0,571.2,Alcoholic cirrhosis of liver,NaN,NaN,0,27653.05,0.0,36 days,30 days,2 days,28 days,10451,0.0,2012-09-01,2012-10-01,30 days,0 days,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6185,HIA|DS1|9809|DS1|NPI|1689608150|2,DS1|9809,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1821297797,1.821298e+09,1689608150,2013-01-20,2013-01-24,0,0,2.0,1,1,21.0,NaN,NaN,I48.0,Paroxysmal atrial fibrillation,0,10721.07,250.0,11 days,13 days,4 days,9 days,9809,0.0,2012-09-01,2012-10-01,30 days,0 days,1
6199,HIA|DS1|9935|DS1|NPI|1902803315|1,DS1|9935,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1972545085,1.972545e+09,1902803315,2011-01-25,2011-01-26,0,0,2.0,4,62,21.0,NaN,NaN,I63.211,Cerebral infarction due to unspecified occlusi...,0,13596.10,250.0,3 days,2 days,1 days,1 days,9935,0.0,2012-09-01,2012-10-01,30 days,0 days,1
6197,HIA|DS1|9935|DS1|NPI|1366401333|1,DS1|9935,DS1|1,DS1|5047X,DS1|NPI|1366401333,DS1|NPI|1790917086,1.790917e+09,1366401333,2011-01-26,2011-02-07,0,0,2.0,D,6,21.0,NaN,NaN,I63.211,Cerebral infarction due to unspecified occlusi...,0,45890.00,250.0,1 days,12 days,12 days,0 days,9935,0.0,2012-09-01,2012-10-01,30 days,0 days,1
6202,HIA|DS1|9943|DS1|NPI|1184722779|2,DS1|9943,DS1|1,DS1|5047X,DS1|NPI|1184722779,DS1|NPI|1942320288,1.942320e+09,1184722779,2013-10-25,2013-10-28,1,0,1.0,1,6,21.0,NaN,NaN,K91.89,Other postprocedural complications and disorde...,0,12277.13,0.0,24 days,12 days,3 days,9 days,9943,0.0,2012-09-01,2012-10-01,30 days,0 days,1


In [20]:
# subset readmissions to prepare to join back to admissions
readmissions2 = readmissions[['Encounter ID', 'readmission']]
readmissions2

,Encounter ID,readmission
45,HIA|DS1|10451|DS1|NPI|1902844988|1,1
37,HIA|DS1|10451|DS1|NPI|1427055839|1,1
38,HIA|DS1|10451|DS1|NPI|1427055839|2,1
40,HIA|DS1|10451|DS1|NPI|1427055839|4,1
41,HIA|DS1|10451|DS1|NPI|1427055839|5,1
...,...,...
6185,HIA|DS1|9809|DS1|NPI|1689608150|2,1
6199,HIA|DS1|9935|DS1|NPI|1902803315|1,1
6197,HIA|DS1|9935|DS1|NPI|1366401333|1,1
6202,HIA|DS1|9943|DS1|NPI|1184722779|2,1


In [21]:
# join readmissions to original admissions
admissions2 = admissions.merge(readmissions2, on='Encounter ID', how='left')
admissions2.head(1)

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,start_date_diff,end_date_diff,length_of_stay,length_between,patient_id_clean,patient_id_diff,ex_date1,ex_date2,thirty_days,zero_days,readmission
0,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1962461012,1.962461e+09,1710918545,2013-06-13,2013-06-16,1,0,1.0,1,1,21.0,NaN,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,29219.0,250.0,NaT,NaT,3 days,NaT,10,NaN,2012-09-01,2012-10-01,30 days,0 days,NaN


In [22]:
# drop extra columns that no longer have meaning
admissions3 = admissions2.drop(['start_date_diff', 'end_date_diff', 'length_of_stay', 'length_between', 'patient_id_diff', 'ex_date1', 'ex_date2', 'thirty_days', 'zero_days'], axis = 1)
admissions3

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,patient_id_clean,readmission
0,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1962461012,1.962461e+09,1710918545,2013-06-13,2013-06-16,1,0,1.0,1,1,21.0,NaN,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,29219.00,250.0,10,NaN
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,0,0,1.0,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,10033,NaN
2,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,0,0,3.0,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,10043,NaN
3,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,1,0,1.0,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,10073,NaN
4,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,0,0,3.0,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,10101,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,0,0,2.0,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,9979,NaN
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,0,1,3.0,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,9981,NaN
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,0,0,1.0,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,9994,NaN
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,0,0,2.0,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,9994,1.0


In [23]:
# calculate readmission rate
admissions3['readmission'].sum() / len(admissions3)

0.1126669885723483

In [24]:
# replace nulls in readmission column with 0
admissions3['readmission'] = admissions3['readmission'].fillna(0)
# result is original admissions dataframe with a column filled with 1s to indcate readmission in <= 30 days and 0s for all other cases
admissions3

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Is Emergency Department Visit Parent Encounter,Is Observation Stay Parent Encounter,CMS Admit Type Code,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,patient_id_clean,readmission
0,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1962461012,1.962461e+09,1710918545,2013-06-13,2013-06-16,1,0,1.0,1,1,21.0,NaN,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,29219.00,250.0,10,0.0
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,0,0,1.0,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,10033,0.0
2,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,0,0,3.0,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,10043,0.0
3,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,1,0,1.0,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,10073,0.0
4,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,0,0,3.0,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,10101,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,0,0,2.0,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,9979,0.0
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,0,1,3.0,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,9981,0.0
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,0,0,1.0,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,9994,0.0
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,0,0,2.0,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,9994,1.0


In [25]:
facility_provider_count = admissions3['Facility Provider ID'].value_counts()
facility_provider_count

DS1|NPI|1902803315    1204
DS1|NPI|1184722779    1196
DS1|NPI|1427055839     889
DS1|NPI|1689772592     679
DS1|NPI|1033298617     460
                      ... 
DS1|NPI|1659538858       1
DS1|NPI|1134274897       1
DS1|NPI|1780757856       1
DS1|NPI|1104842020       1
DS1|NPI|1598744856       1
Name: Facility Provider ID, Length: 220, dtype: int64

In [26]:
facility_provider_readmissions = admissions3.groupby('Facility Provider ID')['readmission'].sum()
facility_provider_readmissions

Facility Provider ID
DS1|NPI|0000000000    17.0
DS1|NPI|1003878539     0.0
DS1|NPI|1003961251     0.0
DS1|NPI|1003980350     0.0
DS1|NPI|1013062769     0.0
                      ... 
DS1|NPI|EI26275047     0.0
DS1|NPI|EI26439240     0.0
DS1|NPI|EI95472322     0.0
DS1|NPI|EI95488125     0.0
DS1|NPI|SY15358485     0.0
Name: readmission, Length: 220, dtype: float64

In [27]:
facility_provider_count = pd.DataFrame(facility_provider_count)
facility_provider_readmissions = facility_provider_readmissions.to_frame()

In [28]:
facility_provider_count.reset_index(inplace=True)
facility_provider_count.rename(columns={'index': 'Facility Provider ID', 'Facility Provider ID': 'admissions'}, inplace=True)
facility_provider_count

,Facility Provider ID,admissions
0,DS1|NPI|1902803315,1204
1,DS1|NPI|1184722779,1196
2,DS1|NPI|1427055839,889
3,DS1|NPI|1689772592,679
4,DS1|NPI|1033298617,460
...,...,...
215,DS1|NPI|1659538858,1
216,DS1|NPI|1134274897,1
217,DS1|NPI|1780757856,1
218,DS1|NPI|1104842020,1


In [29]:
facility_provider_readmissions.reset_index(inplace=True)
facility_provider_readmissions.rename(columns={'readmission': 'readmissions'}, inplace=True)
facility_provider_readmissions

,Facility Provider ID,readmissions
0,DS1|NPI|0000000000,17.0
1,DS1|NPI|1003878539,0.0
2,DS1|NPI|1003961251,0.0
3,DS1|NPI|1003980350,0.0
4,DS1|NPI|1013062769,0.0
...,...,...
215,DS1|NPI|EI26275047,0.0
216,DS1|NPI|EI26439240,0.0
217,DS1|NPI|EI95472322,0.0
218,DS1|NPI|EI95488125,0.0


In [30]:
facility_providers = facility_provider_count.merge(facility_provider_readmissions, on='Facility Provider ID', how='left')
facility_providers

,Facility Provider ID,admissions,readmissions
0,DS1|NPI|1902803315,1204,98.0
1,DS1|NPI|1184722779,1196,113.0
2,DS1|NPI|1427055839,889,115.0
3,DS1|NPI|1689772592,679,82.0
4,DS1|NPI|1033298617,460,44.0
...,...,...,...
215,DS1|NPI|1659538858,1,0.0
216,DS1|NPI|1134274897,1,0.0
217,DS1|NPI|1780757856,1,0.0
218,DS1|NPI|1104842020,1,0.0


In [31]:
facility_providers['readmission_rate'] = facility_providers['readmissions'] / facility_providers['admissions']
facility_providers.sort_values('readmission_rate', inplace=True, ascending=False)
facility_providers

,Facility Provider ID,admissions,readmissions,readmission_rate
58,DS1|NPI|1639172133,4,4.0,1.0
197,DS1|NPI|1457347239,1,1.0,1.0
174,DS1|NPI|1063441293,1,1.0,1.0
163,DS1|NPI|1376698043,1,1.0,1.0
34,DS1|NPI|1700804200,8,8.0,1.0
...,...,...,...,...
108,DS1|NPI|1912914821,2,0.0,0.0
109,DS1|NPI|1952445959,2,0.0,0.0
111,DS1|NPI|1508859323,2,0.0,0.0
112,DS1|NPI|1043263080,1,0.0,0.0


In [32]:
facility_providers2 = facility_providers[facility_providers['admissions'] >=20]
facility_providers2

,Facility Provider ID,admissions,readmissions,readmission_rate
14,DS1|NPI|1639172372,20,5.0,0.250000
7,DS1|NPI|1710065933,109,24.0,0.220183
13,DS1|NPI|1174667158,21,4.0,0.190476
8,DS1|NPI|0000000000,103,17.0,0.165049
12,DS1|NPI|1811080526,22,3.0,0.136364
2,DS1|NPI|1427055839,889,115.0,0.129359
6,DS1|NPI|1689608150,340,43.0,0.126471
9,DS1|NPI|1992818256,87,11.0,0.126437
3,DS1|NPI|1689772592,679,82.0,0.120766
10,DS1|NPI|1154428688,27,3.0,0.111111


In [33]:
patient_count = admissions3['Patient ID'].value_counts()
patient_count

DS1|3812     17
DS1|26751    15
DS1|53241    13
DS1|10451    12
DS1|21926    11
             ..
DS1|31911     1
DS1|36994     1
DS1|48301     1
DS1|55779     1
DS1|35119     1
Name: Patient ID, Length: 4701, dtype: int64

In [34]:
patient_readmissions = admissions3.groupby('Patient ID')['readmission'].sum()
patient_readmissions

Patient ID
DS1|10       0.0
DS1|10033    0.0
DS1|10043    0.0
DS1|10073    0.0
DS1|10101    0.0
            ... 
DS1|9978     0.0
DS1|9979     0.0
DS1|9981     0.0
DS1|9994     1.0
DS1|9996     0.0
Name: readmission, Length: 4701, dtype: float64

In [35]:
patient_count = pd.DataFrame(patient_count)
patient_readmissions = patient_readmissions.to_frame()

In [36]:
patient_count.reset_index(inplace=True)
patient_count.rename(columns={'index': 'Patient ID', 'Patient ID': 'admissions'}, inplace=True)
patient_count

,Patient ID,admissions
0,DS1|3812,17
1,DS1|26751,15
2,DS1|53241,13
3,DS1|10451,12
4,DS1|21926,11
...,...,...
4696,DS1|31911,1
4697,DS1|36994,1
4698,DS1|48301,1
4699,DS1|55779,1


In [37]:
patient_readmissions.reset_index(inplace=True)
patient_readmissions.rename(columns={'readmission': 'readmissions'}, inplace=True)
patient_readmissions

,Patient ID,readmissions
0,DS1|10,0.0
1,DS1|10033,0.0
2,DS1|10043,0.0
3,DS1|10073,0.0
4,DS1|10101,0.0
...,...,...
4696,DS1|9978,0.0
4697,DS1|9979,0.0
4698,DS1|9981,0.0
4699,DS1|9994,1.0


In [38]:
patients = patient_count.merge(patient_readmissions, on='Patient ID', how='left')
patients

,Patient ID,admissions,readmissions
0,DS1|3812,17,8.0
1,DS1|26751,15,5.0
2,DS1|53241,13,9.0
3,DS1|10451,12,7.0
4,DS1|21926,11,10.0
...,...,...,...
4696,DS1|31911,1,0.0
4697,DS1|36994,1,0.0
4698,DS1|48301,1,0.0
4699,DS1|55779,1,0.0


In [39]:
patients['readmission_rate'] = patients['readmissions'] / patients['admissions']
patients.sort_values('readmission_rate', inplace=True, ascending=False)
patients.head(30)

,Patient ID,admissions,readmissions,readmission_rate
4,DS1|21926,11,10.0,0.909091
6,DS1|12422,11,10.0,0.909091
11,DS1|13593,9,8.0,0.888889
20,DS1|3663,8,7.0,0.875000
27,DS1|54840,7,6.0,0.857143
37,DS1|37954,6,5.0,0.833333
61,DS1|35155,5,4.0,0.800000
50,DS1|19074,5,4.0,0.800000
69,DS1|39480,5,4.0,0.800000
52,DS1|30393,5,4.0,0.800000


In [49]:
primary

,Encounter ID,Patient ID,Service Provider ID,Attending Provider ID,Provider Group Provider ID,Facility Provider ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI,Encounter Date,Is Qualifying Preventive Primary Care Service,Is Qualifying Primary Care Service Provider,Is Qualifying Primary Care Attending,Is Qualifying Primary Care Provider Group,Qualifying Preventive Primary Care HCPCS Code
0,COV|EE|DS1|10076|20110225|DS1|NPI|1487856779,DS1|10076,DS1|NPI|1487856779,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.487857e+09,1.043427e+09,2011-02-18,0,1,0,0,NaN
1,COV|EE|DS1|10088|20140222|DS1|NPI|1528030996,DS1|10088,DS1|NPI|1528030996,DS1|NPI|1528030996,DS1|NPI|1043427248,DS1|NPI|1902803315,DS1|1,DS1|5047W,1.528031e+09,1.902803e+09,1.528031e+09,1.043427e+09,2014-02-22,0,1,1,1,NaN
2,COV|EE|DS1|10165|20140907|DS1|NPI|1477579365,DS1|10165,DS1|NPI|1477579365,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.477579e+09,1.043427e+09,2014-08-31,0,1,0,0,NaN
3,COV|EE|DS1|10240|20130624|DS1|NPI|1821178088,DS1|10240,DS1|NPI|1821178088,DS1|NPI|1821178088,DS1|NPI|1407058787,DS1|NPI|1104842020,DS1|1,DS1|5047W,1.821178e+09,1.104842e+09,1.821178e+09,1.407059e+09,2013-06-17,0,1,1,1,NaN
4,COV|EE|DS1|10293|20160904|DS1|NPI|1609004191,DS1|10293,DS1|NPI|1609004191,NaN,DS1|NPI|1184722779,NaN,DS1|1,DS1|5047X,NaN,NaN,1.609004e+09,1.184723e+09,2016-09-04,0,1,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231368,COV|PCSL|DS1|9999|P|91236|1184722779|1,DS1|9999,DS1|NPI|1851738108,NaN,DS1|NPI|1184722779,NaN,DS1|1,DS1|5047X,NaN,NaN,1.851738e+09,1.184723e+09,2015-02-10,0,1,0,0,NaN
231369,COV|PCSL|DS1|99|P|1263621|1043427248|1,DS1|99,DS1|NPI|1215163035,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.215163e+09,1.043427e+09,2011-08-02,0,1,0,0,NaN
231370,COV|PCSL|DS1|99|P|22491|1043427248|1,DS1|99,DS1|NPI|1215163035,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.215163e+09,1.043427e+09,2011-04-13,1,1,0,0,99385.0
231371,COV|PCSL|DS1|9|P|341904|1962790014|1,DS1|9,DS1|NPI|1144252396,NaN,DS1|NPI|1962790014,NaN,DS1|1,DS1|5047X,NaN,NaN,1.144252e+09,1.962790e+09,2014-04-07,0,1,0,0,NaN


In [43]:
primary['Encounter Date'] = pd.to_datetime(primary['Encounter Date'])

In [45]:
primary['Encounter Date'].max()

Timestamp('2019-09-22 00:00:00')

In [54]:
# filter primary and admissions to prepare to concatenatee
primary2 = primary[['Encounter ID', 'Patient ID', 'Facility Provider ID', 'Encounter Date']]
primary2.rename(columns={'Encounter Date': 'Encounter End Date'}, inplace=True)
# identify all primary care visits as type 1
primary2['type'] = 1
admissions4 = admissions[['Encounter ID', 'Patient ID', 'Facility Provider ID', 'Encounter End Date']]
# identify all hospital visits as 0
admissions['type'] = 0
hospital_and_primary = pd.concat([admissions4, primary2], ignore_index=True)
# identify all hospital visits as 0 again b/c showed up as null the first time?
hospital_and_primary['type'] = hospital_and_primary['type'].fillna(0)
hospital_and_primary

C:\Users\rhilb\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-54-e3a1f94065df>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary2['type'] = 1


,Encounter ID,Patient ID,Facility Provider ID,Encounter End Date,type
0,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|NPI|1710918545,2013-06-16,0.0
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|NPI|1306890389,2013-01-14,0.0
2,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|NPI|1902803315,2014-01-12,0.0
3,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|NPI|1902803315,2012-06-21,0.0
4,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|NPI|1427055839,2012-12-09,0.0
...,...,...,...,...,...
237581,COV|PCSL|DS1|9999|P|91236|1184722779|1,DS1|9999,NaN,2015-02-10,1.0
237582,COV|PCSL|DS1|99|P|1263621|1043427248|1,DS1|99,NaN,2011-08-02,1.0
237583,COV|PCSL|DS1|99|P|22491|1043427248|1,DS1|99,NaN,2011-04-13,1.0
237584,COV|PCSL|DS1|9|P|341904|1962790014|1,DS1|9,NaN,2014-04-07,1.0


In [55]:
# sort by patient id and date so that patients are grouped and in order by date
hospital_and_primary.sort_values(["Patient ID", 'Encounter End Date'], inplace=True)
hospital_and_primary

,Encounter ID,Patient ID,Facility Provider ID,Encounter End Date,type
82145,COV|PCSL|DS1|1|P|10130|1083694616|1,DS1|1,NaN,2013-04-11,1.0
82146,COV|PCSL|DS1|1|P|371880|1083694616|1,DS1|1,NaN,2013-11-04,1.0
82147,COV|PCSL|DS1|1|P|659989|1083694616|1,DS1|1,NaN,2013-11-21,1.0
50001,COV|PBSL|DS1|10|P|649805|1144213166|1,DS1|10,NaN,2011-08-17,1.0
50002,COV|PBSL|DS1|10|P|702882|1144213166|1,DS1|10,NaN,2011-08-23,1.0
...,...,...,...,...,...
237576,COV|PCSL|DS1|9999|P|1116342|1043427248|1,DS1|9999,NaN,2011-09-02,1.0
237579,COV|PCSL|DS1|9999|P|642185|1043427248|1,DS1|9999,NaN,2012-11-04,1.0
237578,COV|PCSL|DS1|9999|P|494636|1184722779|1,DS1|9999,NaN,2014-10-07,1.0
237580,COV|PCSL|DS1|9999|P|818335|1184722779|1,DS1|9999,NaN,2014-12-09,1.0


In [56]:
# clean patient ids
patient_id2 = hospital_and_primary['Patient ID'].str.split('|', expand=True)
patient_id2

,0,1
82145,DS1,1
82146,DS1,1
82147,DS1,1
50001,DS1,10
50002,DS1,10
...,...,...
237576,DS1,9999
237579,DS1,9999
237578,DS1,9999
237580,DS1,9999


In [57]:
hospital_and_primary['patient_id_clean'] = patient_id2[1]
hospital_and_primary

,Encounter ID,Patient ID,Facility Provider ID,Encounter End Date,type,patient_id_clean
82145,COV|PCSL|DS1|1|P|10130|1083694616|1,DS1|1,NaN,2013-04-11,1.0,1
82146,COV|PCSL|DS1|1|P|371880|1083694616|1,DS1|1,NaN,2013-11-04,1.0,1
82147,COV|PCSL|DS1|1|P|659989|1083694616|1,DS1|1,NaN,2013-11-21,1.0,1
50001,COV|PBSL|DS1|10|P|649805|1144213166|1,DS1|10,NaN,2011-08-17,1.0,10
50002,COV|PBSL|DS1|10|P|702882|1144213166|1,DS1|10,NaN,2011-08-23,1.0,10
...,...,...,...,...,...,...
237576,COV|PCSL|DS1|9999|P|1116342|1043427248|1,DS1|9999,NaN,2011-09-02,1.0,9999
237579,COV|PCSL|DS1|9999|P|642185|1043427248|1,DS1|9999,NaN,2012-11-04,1.0,9999
237578,COV|PCSL|DS1|9999|P|494636|1184722779|1,DS1|9999,NaN,2014-10-07,1.0,9999
237580,COV|PCSL|DS1|9999|P|818335|1184722779|1,DS1|9999,NaN,2014-12-09,1.0,9999


In [58]:
hospital_and_primary['patient_id_clean'] = pd.to_numeric(hospital_and_primary['patient_id_clean'])

In [59]:
# calculate difference between subsequent dates
hospital_and_primary['date_diff'] = hospital_and_primary['Encounter End Date'].diff()
# calculate difference between subsequent types 
hospital_and_primary['type_diff'] = hospital_and_primary['type'].diff()
# calculate difference between subsequent patient ids
hospital_and_primary['patient_id_diff'] = hospital_and_primary['patient_id_clean'].diff()

In [60]:
hospital_and_primary

,Encounter ID,Patient ID,Facility Provider ID,Encounter End Date,type,patient_id_clean,date_diff,type_diff,patient_id_diff
82145,COV|PCSL|DS1|1|P|10130|1083694616|1,DS1|1,NaN,2013-04-11,1.0,1,NaT,NaN,NaN
82146,COV|PCSL|DS1|1|P|371880|1083694616|1,DS1|1,NaN,2013-11-04,1.0,1,207 days,0.0,0.0
82147,COV|PCSL|DS1|1|P|659989|1083694616|1,DS1|1,NaN,2013-11-21,1.0,1,17 days,0.0,0.0
50001,COV|PBSL|DS1|10|P|649805|1144213166|1,DS1|10,NaN,2011-08-17,1.0,10,-827 days,0.0,9.0
50002,COV|PBSL|DS1|10|P|702882|1144213166|1,DS1|10,NaN,2011-08-23,1.0,10,6 days,0.0,0.0
...,...,...,...,...,...,...,...,...,...
237576,COV|PCSL|DS1|9999|P|1116342|1043427248|1,DS1|9999,NaN,2011-09-02,1.0,9999,133 days,0.0,0.0
237579,COV|PCSL|DS1|9999|P|642185|1043427248|1,DS1|9999,NaN,2012-11-04,1.0,9999,429 days,0.0,0.0
237578,COV|PCSL|DS1|9999|P|494636|1184722779|1,DS1|9999,NaN,2014-10-07,1.0,9999,702 days,0.0,0.0
237580,COV|PCSL|DS1|9999|P|818335|1184722779|1,DS1|9999,NaN,2014-12-09,1.0,9999,63 days,0.0,0.0


In [63]:
# filter for only patient id = 0 to only include rows for repeated patients
sees_pcp = hospital_and_primary[hospital_and_primary['patient_id_diff'] == 0]
# filter for type diff equals 1 which will only include pcp visits right after hospital visits
sees_pcp = sees_pcp[sees_pcp['type_diff'] == 1]
#filter for date differences less than or equal to 7
sees_pcp = sees_pcp[sees_pcp['date_diff'] <= dt.timedelta(days = 7)]
sees_pcp

,Encounter ID,Patient ID,Facility Provider ID,Encounter End Date,type,patient_id_clean,date_diff,type_diff,patient_id_diff
46604,COV|PBSL|DS1|10043|P|538933|1043427248|1|SHIFTED,DS1|10043,NaN,2014-01-12,1.0,10043,0 days,1.0,0.0
46686,COV|PBSL|DS1|10073|P|301709|1043427248|1|SHIFTED,DS1|10073,NaN,2012-06-27,1.0,10073,6 days,1.0,0.0
52861,COV|PCSL|DS1|10101|P|1189971|1043427248|1,DS1|10101,NaN,2012-12-12,1.0,10101,3 days,1.0,0.0
47619,COV|PBSL|DS1|10303|P|924098|1912987553|1,DS1|10303,NaN,2012-06-25,1.0,10303,1 days,1.0,0.0
47628,COV|PBSL|DS1|10305|P|823298|1184722779|1,DS1|10305,NaN,2014-05-13,1.0,10305,1 days,1.0,0.0
...,...,...,...,...,...,...,...,...,...
237404,COV|PCSL|DS1|9943|P|1315984|1184722779|1,DS1|9943,NaN,2014-11-03,1.0,9943,4 days,1.0,0.0
237417,COV|PCSL|DS1|9945|P|840374|1043427248|1,DS1|9945,NaN,2013-02-14,1.0,9945,3 days,1.0,0.0
237516,COV|PCSL|DS1|9978|P|696226|1043427248|1,DS1|9978,NaN,2015-09-11,1.0,9978,6 days,1.0,0.0
237520,COV|PCSL|DS1|9979|P|281365|1477666352|1,DS1|9979,NaN,2015-02-15,1.0,9979,4 days,1.0,0.0


In [64]:
# calculate rate of seeing pcp within 7 days of hospital visit
len(sees_pcp) / len(admissions)

0.2974408498309995

In [65]:
sees_pcp['date_diff'].max()

Timedelta('7 days 00:00:00')

In [66]:
sees_pcp['sees_pcp'] = 1

In [67]:
sees_pcp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1848 entries, 46604 to 237524
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   Encounter ID          1848 non-null   object         
 1   Patient ID            1848 non-null   object         
 2   Facility Provider ID  242 non-null    object         
 3   Encounter End Date    1848 non-null   datetime64[ns] 
 4   type                  1848 non-null   float64        
 5   patient_id_clean      1848 non-null   int64          
 6   date_diff             1848 non-null   timedelta64[ns]
 7   type_diff             1848 non-null   float64        
 8   patient_id_diff       1848 non-null   float64        
 9   sees_pcp              1848 non-null   int64          
dtypes: datetime64[ns](1), float64(3), int64(2), object(3), timedelta64[ns](1)
memory usage: 158.8+ KB
